In [38]:
## data collection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

## load dataset

data = gutenberg.raw('shakespeare-hamlet.txt')
## save to a file

with open('hamlet.txt','w')as file:
    file.write(data)


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [39]:
##  data preprocessing

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


## load the dataset

with open('hamlet.txt','r')as file:
    text=file.read().lower()

## tokenizer the text
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
total_words

4818

In [40]:
## create input sequences
inputsequences=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        inputsequences.append(n_gram_sequence)


In [41]:
## pad sequences

max_sequence_len = max([len(x) for x in inputsequences])
max_sequence_len

14

In [42]:
inputsequences=np.array(pad_sequences(inputsequences,maxlen=max_sequence_len,padding='pre'))
inputsequences


array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [43]:
# create predictors and label

import tensorflow as tf
X,y = inputsequences[:,:-1],inputsequences[:,-1]


In [44]:
y = tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [45]:
## split the data int training and testing

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)


In [46]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping=EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)

In [47]:
## train our LSTM rnn

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,GRU


## define the model
model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(GRU(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(100))
model.add(Dense(total_words,activation='softmax'))

## compile the model




/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [48]:
model.build(input_shape=(None, max_sequence_len))
model.summary()
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 14, 100)             │         481,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 14, 150)             │         113,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 14, 150)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 100)                 │          75,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4818)                │         486,618 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,157,418 (4.42 MB)

 Trainable params: 1,157,418 (4.42 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
## train the model
history = model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),verbose=1)

Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 44s 60ms/step - accuracy: 0.0292 - loss: 7.1666 - val_accuracy: 0.0394 - val_loss: 6.8678
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 40s 59ms/step - accuracy: 0.0420 - loss: 6.4220 - val_accuracy: 0.0538 - val_loss: 6.8200
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 42s 61ms/step - accuracy: 0.0553 - loss: 6.1361 - val_accuracy: 0.0688 - val_loss: 6.7845
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 42s 62ms/step - accuracy: 0.0739 - loss: 5.8592 - val_accuracy: 0.0756 - val_loss: 6.8716
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 40s 61ms/step - accuracy: 0.0835 - loss: 5.5668 - val_accuracy: 0.0771 - val_loss: 6.9443
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - accuracy: 0.0955 - loss: 5.2656 - val_accuracy: 0.0736 - val_loss: 7.0132
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 39s 61ms/step - accuracy: 0.1074 - loss: 5.0218 - val_accuracy: 0.0748 - val_loss: 7.1550
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 41s 64ms/step - accuracy: 0.1235 - loss: 4

In [50]:
def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list)>=max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]
    token_list = pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted = model.predict(token_list,verbose=0)
    predicted_word_index = np.argmax(predicted,axis=1)
    for word,index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [51]:
input_text = "They are so fortified against your"
print(f'Input text:{input_text}')
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next word prediction: {next_word}")

Input text:They are so fortified against your
Next word prediction: story


In [52]:
## model save
model.save("next_word_lstm.h5")

## save the tokenizer
import pickle
with open('tokenzer.pickle','wb')as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)